In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
%matplotlib inline

In [2]:
matplotlib.rcParams['figure.figsize'] = [10, 5]

# Engineeering features


En este notebook vamos a ver principalmente:

1) Tratamiento de NAs values

2) Tratamiento de variables categóricas


Además de esto, hablamos de ingeniería de variables a todas las modificaciones que se hacen de las variables disponibles. Estas modificaciones suelen ser muy naturales si se entiende muy bien el dataset
y se han detectado algunos patrones o relaciones con la variable a predecir. Por ejemplo, 

- para fechas: pasando estas a semana, dia, mes, quarter, etc...domnde descubrimos un patron o seasonality.

- para texto: donde su longitud o sus palabras repetidas podrían dar un valor clave.

Vamos a trabajar con el asiguiente dataset llamado __House Prices__ que puedes encontrar
[aquí](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)

### 1. Bajamos el dataset y hacemos un simple modelo  (Regresión multilineal)


In [3]:
path = "./data/house-prices-advanced-regression-techniques/train.csv"
houses = pd.read_csv(path)

In [4]:
houses_num =  houses.select_dtypes(include=[np.number])

In [5]:
houses_num

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
5,6,50,85.0,14115,5,5,1993,1995,0.0,732,...,40,30,0,320,0,0,700,10,2009,143000
6,7,20,75.0,10084,8,5,2004,2005,186.0,1369,...,255,57,0,0,0,0,0,8,2007,307000
7,8,60,NaN,10382,7,6,1973,1973,240.0,859,...,235,204,228,0,0,0,350,11,2009,200000
8,9,50,51.0,6120,7,5,1931,1950,0.0,0,...,90,0,205,0,0,0,0,4,2008,129900
9,10,190,50.0,7420,5,6,1939,1950,0.0,851,...,0,4,0,0,0,0,0,1,2008,118000


#### Problema:  ¿Podemos predecir el precio de venta?

Hagamos una regresión multilineal con todas las variables numéricas

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

target_col = "SalePrice"
Xy = houses_num
X = Xy.drop(target_col, axis=1)
y = Xy[target_col]

X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_dev_hat = reg.predict(X_dev)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

__¿Qué ha pasado?__

```
ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
```

### 2. Tratamiento de Nas

##### Busquemos nuestros NAs

In [7]:
houses_num =  houses.select_dtypes(include=[np.number])
houses_num.shape

(1460, 38)

In [8]:
houses_num.isnull().sum()

Id                 0
MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
SalePrice          0
dtype: int64

###### Método 1: eliminar las columnas con NAs

In [9]:
columnswithNulls = ['LotFrontage','GarageYrBlt',"MasVnrArea"]

In [10]:
houses_num.drop(columnswithNulls,axis=1).isnull().sum().sum()

0

In [11]:
target_col = "SalePrice"
Xy = houses_num.drop(columnswithNulls,axis=1)
X = Xy.drop(target_col, axis=1)
y = Xy[target_col]

X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_dev_hat = reg.predict(X_dev)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

34461.69716294582


###### Método 2: eliminar las filas con NAs

In [12]:
Xy = houses_num
print(Xy.shape)
Xy = Xy.dropna()
print(Xy.shape)

(1460, 38)
(1121, 38)


In [13]:
X = Xy.drop(target_col, axis=1).dropna()
y = Xy[target_col]
X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_dev_hat = reg.predict(X_dev)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

SystemError: Failed to solve the system of normal equations


###### Método 3: sustituir NAs por valores adecuados

La idea es sustituir por valores que entran dentro del dominio de 
la variable; por ejemplo 0, un valor aleatorio, una media, una mediana, moda o incluso un valor estimado por otro modelo predictivo.

In [14]:
columnswithNulls = ['LotFrontage','GarageYrBlt',"MasVnrArea"]

In [15]:
houses[columnswithNulls].describe()

,LotFrontage,GarageYrBlt,MasVnrArea
count,1201.000000,1379.000000,1452.000000
mean,70.049958,1978.506164,103.685262
std,24.284752,24.689725,181.066207
min,21.000000,1900.000000,0.000000
25%,59.000000,1961.000000,0.000000
50%,69.000000,1980.000000,0.000000
75%,80.000000,2002.000000,166.000000
max,313.000000,2010.000000,1600.000000


##### Por un valor aleatorio

In [16]:
import random
aux = houses_num.copy()
aux[columnswithNulls].isnull().sum().sum()

348

In [17]:
for col in columnswithNulls:
    minVal =  aux[col].min()
    maxVal = aux[col].max()
    aux.loc[aux[col].isnull(),col] =  random.randrange(minVal,maxVal)

In [18]:
X = aux.drop(target_col, axis=1).dropna()
y = aux[target_col]
X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_dev_hat = reg.predict(X_dev)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

SystemError: Failed to solve the system of normal equations


##### Por la media

In [ ]:
aux = houses_num.copy()
aux[columnswithNulls].isnull().sum().sum()

In [ ]:
for col in columnswithNulls:
    meanVal =  aux[col].mean()
    aux.loc[aux[col].isnull(),col] =  meanVal

In [ ]:
X = aux.drop(target_col, axis=1).dropna()
y = aux[target_col]
X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_dev_hat = reg.predict(X_dev)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

##### Por el mínimo

In [ ]:
aux = houses_num.copy()
for col in columnswithNulls:
    minVal =  aux[col].min()
    aux.loc[aux[col].isnull(),col] =  minVal

In [ ]:
aux[columnswithNulls].isnull().sum().sum()

In [ ]:
X = aux.drop(target_col, axis=1).dropna()
y = aux[target_col]
X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_dev_hat = reg.predict(X_dev)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

> __Nota__ : no todos los algoritmos fallan cuando faltan datos. Hay algoritmos que pueden ser robustos a NAs, como k-Nearest Neighbors que pueden ignorar una columna desde una medida de distancia cuando falta un valor; también hay algoritmos que pueden usar el valor faltante como un valor único y diferente al construir el modelo predictivo, como los árboles de clasificación y regresión.

### 2. Variables categóricas: tratamiento

En numerosas ocasiones nos enfrentaremos a variables categóricas las cuales tienen una fuerte correlación con la variable a predecir. Por ejemplo en el caso del predcio de venta de casas, claramente el tipo de barrio es un variable totalmente importante para predecir su precio. Un casa de msimas carazterísticas no tiene el mismo valor en Extremadura que en Barcelona. Veremos aquí qué metodologías existen para tratar variables categóricas:

a) Labelizar

2) One-hot-encoding

Busquemos en nuestro dataset quienes son las variables categóricas

In [ ]:
houses_cat =  houses.select_dtypes(include=[np.object])

In [ ]:
houses_cat.shape

In [ ]:
houses_cat.describe()

In [ ]:
houses_cat['MSZoning'].value_counts()

### 2.1 Labelización

In [ ]:
houses_cat['SaleCondition'].value_counts()

La idea de labelizar es darle un valor a cada uno:

Normal: 0
Partial: 1
Abnorml: 2
Family:3
Alloca: 4
AdjLand: 5


In [ ]:
categories = houses_cat['SaleCondition'].value_counts().index.values
categories

In [ ]:
col2num = {categories[i]: i for i in range(len(categories))}
col2num

In [ ]:
aux['SaleConditionNum'] = houses_cat['SaleCondition'].apply(lambda cls: col2num[cls])

In [ ]:
X = aux.drop(target_col, axis=1).dropna()
y = aux[target_col]
X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_dev_hat = reg.predict(X_dev)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

> __Problema__ : El problema más importante con este tipo de codificación es que no hay un orden natural de esas categorías. Lo veremos con el siguiente ejemplo usando Neiborhood

Otro ejemplo:

In [ ]:
houses = pd.read_csv(path, index_col="Id")

target_col = "SalePrice"
categorical_column = "Neighborhood"
Xy = houses[[categorical_column, target_col]]
X = Xy.drop(target_col, axis=1)
y = Xy[target_col]

X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

In [ ]:
classes = X[categorical_column].unique()
col2num = {classes[i]: i for i in range(len(classes))}
X_train[categorical_column + "_label"] = X_train[categorical_column].apply(lambda cls: col2num[cls])
X_dev[categorical_column + "_label"] = X_dev[categorical_column].apply(lambda cls: col2num[cls])

In [ ]:
reg = LinearRegression()
reg.fit(X_train[[categorical_column + "_label"]], y_train)
y_dev_hat = reg.predict(X_dev[[categorical_column + "_label"]])
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

In [ ]:
# Ploting real data y (blue)
plt.scatter(X_dev[[categorical_column + "_label"]], y_dev, alpha=0.3, c="blue")
# versus predicted y_hat (red)


plt.plot(X_dev[[categorical_column + "_label"]], y_dev_hat, alpha=0.3, c="red")

plt.xlabel("Labelized category")
plt.ylabel("Sale Price")

__Nota__ : Vemos que la predicción es casi constante y probablemente deberíamos reordenar las etiquetas de una manera diferente. Hay dos maneras de resolverlo que discutiremos aquí.

1. Podríamos etiquetar las categorías de manera que no tengan un pedido. Podemos hacer esto usando One-Hot-Encoding.

2. Categorías etiquetadas por números que tiene sentido.

### 2.1 One-hot-encoding

Esto es creando tantas columnas como categorías que tenemos y rellenándolas con ceros, excepto la que corresponde a la categoría. Así codifica las categorías de la siguiente manera:

* dog to `[1, 0, 1]`
* cat to `[0, 1, 0]`
* horse to `[0, 0, 1]`

en el caso de una variable con tres valores distintos

In [ ]:
def label_one_hot_encoding(series, dict_length=None):
    if dict_length is None:
        dict_length = len(series.unique())
    return np.eye(dict_length)[series]

Vamos a ver la diferencia con la labelización para el mismo ejemplo:

In [ ]:
dict_length = len(X_train[categorical_column + "_label"].unique())

In [ ]:
reg.fit(label_one_hot_encoding(X_train[categorical_column + "_label"], dict_length), y_train)
y_dev_hat = reg.predict(label_one_hot_encoding(X_dev[categorical_column + "_label"], dict_length))
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

### 2.1 Value encoding

Otra forma de codificar categorías es usar valores promedio de la variable objetivo. En realidad, en esta situación obtendremos exactamente las mismas predicciones `y_dev_hat`. Lo más importante que debes recordar es utilizar `X_train` y ` y_train`. Veamos con el siguiente código lo que significa.

In [ ]:
category_value_df = X_train[[categorical_column]]
category_value_df[categorical_column+"_value"] = y_train
category_value_df = category_value_df.groupby(categorical_column, as_index=False).mean()
category_value_df.head()

In [ ]:
y_dev_hat_value_encoded = \
    X_dev.merge(category_value_df, how="left", on=categorical_column)[[categorical_column+"_value"]]
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

    Poniéndolo todo junto:

In [ ]:
columns = ['OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 
'TotalBsmtSF', "Neighborhood", "ExterQual"]

target_col = "SalePrice"

X = houses[columns]
y = houses[target_col]

X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

In [ ]:
class Labelizer:
    
    def __init__(self, categories):
        self.categories = categories
        self.cat2num = {}
        self.num2cat = {}
        
    def generate_dicts(self):
        self.cat2num = {self.categories[i]: i for i in range(len(self.categories))}
        self.num2cat = {i: self.categories[i] for i in range(len(self.categories))}
        
    def fit(self):
        self.generate_dicts()
        return self
        
    def transform(self, series):
        return series.apply(lambda cat: self.cat2num[cat])
    
labelizer_Neighborhood = Labelizer(X_train["Neighborhood"].unique()).fit()  
labelizer_ExterQual = Labelizer(X_train["ExterQual"].unique()).fit()

def labelize_X(X):
    labelized_X = X.drop(["Neighborhood", "ExterQual"], axis=1)
    labelized_X["Neighborhood_labelized"] = labelizer_Neighborhood.transform(X["Neighborhood"])
    labelized_X["ExterQual_labelized"] = labelizer_ExterQual.transform(X["ExterQual"])
    return labelized_X

X_train_labelized = labelize_X(X_train)
X_dev_labelized = labelize_X(X_dev)

reg.fit(X_train_labelized, y_train)
y_dev_hat = reg.predict(X_dev_labelized)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

In [ ]:
A = np.array([[1, 2], [3, 4], [5, 6]])
B = np.array([[7], [8], [9]])
C = np.array([[11, 12], [13, 14], [16, 15]])
np.concatenate([A, B, C], axis=1)

In [ ]:
class OneHotEncoderWithLabelizer:
    
    def __init__(self, categories):
        self.categories = categories
        self.labelizer = Labelizer(categories)
            
    def fit(self):
        self.labelizer.fit()
        return self
        
    def transform(self, series):
        series_labelized = self.labelizer.transform(series)
        return np.eye(len(self.categories))[series_labelized]
    
one_hot_encoder_Neighborhood = OneHotEncoderWithLabelizer(X_train["Neighborhood"].unique()).fit()  
one_hot_encoder_ExterQual = OneHotEncoderWithLabelizer(X_train["ExterQual"].unique()).fit()

def one_hot_encode_X(X):
    X_other = X.drop(["Neighborhood", "ExterQual"], axis=1).as_matrix()
    neighborhood_encoded = one_hot_encoder_Neighborhood.transform(X["Neighborhood"])
    exterQual_encoded = one_hot_encoder_ExterQual.transform(X["ExterQual"])
    return np.concatenate([X_other, neighborhood_encoded, exterQual_encoded], axis=1)

X_train_encoded = one_hot_encode_X(X_train)
X_dev_encoded = one_hot_encode_X(X_dev)

reg.fit(X_train_encoded, y_train)
y_dev_hat = reg.predict(X_dev_encoded)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

In [ ]:
import string

class ValueEncoder:
    
    def __init__(self, cat_colname=None, value_colname=None, agg_function='mean',):
        self.agg_function = agg_function
        self.cat_colname = cat_colname if cat_colname is not None else\
            ''.join(np.random.choice(list(string.ascii_lowercase + string.digits)) for _ in range(10))
        self.value_colname = value_colname if value_colname is not None else\
            self.cat_colname + "_" + self.agg_function
        self.cat_value_df = None
            
    def fit(self, cat_series, value_series):
        cat_value_df = pd.DataFrame({
            self.cat_colname: cat_series,
            self.value_colname: value_series
        })
        self.cat_value_df = cat_value_df\
            .groupby(self.cat_colname, as_index=False)\
            .agg(self.agg_function)
        return self
        
    def transform(self, cat_series):
        return pd.DataFrame({self.cat_colname:cat_series})\
            .merge(self.cat_value_df, on=self.cat_colname, how="left")[self.value_colname]

value_encoder_Neighborhood = ValueEncoder("Neighborhood")\
   .fit(X_train["Neighborhood"], y_train)  
value_encoder_ExterQual = ValueEncoder("ExterQual")\
   .fit(X_train["ExterQual"], y_train)

def value_encode_X(X):
    encoded_X = X.drop(["Neighborhood", "ExterQual"], axis=1).reset_index(drop=True)
    encoded_X["Neighborhood_encoded"] = value_encoder_Neighborhood.transform(X["Neighborhood"])
    encoded_X["ExterQual_encoded"] = value_encoder_ExterQual.transform(X["ExterQual"])
    return encoded_X

X_train_encoded = value_encode_X(X_train)
X_dev_encoded = value_encode_X(X_dev)
reg.fit(X_train_encoded, y_train)
y_dev_hat = reg.predict(X_dev_encoded)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))